In [20]:
import pandas as pd

data = pd.read_csv(r'RawData/cvn-hvac-supply-data.csv') #35194 x 23. 14330 unique job_seq

# Variables Already Present in Jobs Dataset
<pre>
job_seq: Will be used for joining purposes
jcn
uic
work_center
date_maintenance_action
date_closing
issue_apl
issue_eic
<pre>

# Variables Not Present in Jobs Dataset
<pre>
demand_date
issue_date
cwt_hours
niin
niin_nomenclature
unit_of_issue
supply_pri
source_code
quantity
unit_price
total_price
<pre>

# Removing Unnecessary Columns

In [21]:
#Dimension is now 35194 x 18
data = data.drop(['document_number', 'request_num', 'requisition_number', 'cwt_hours', 'total_price'], axis=1) 

# NA Handling

In [22]:
na_counts = data.isnull().sum()

print(f"{'Column Name':<40} | {'# of NAs'}")
print("-" * 40 + " | " + "-" * 8)

for col, count in na_counts.items():
    print(f"{col:<40} | {count}")

Column Name                              | # of NAs
---------------------------------------- | --------
jcn                                      | 0
uic                                      | 0
work_center                              | 0
jsn                                      | 0
date_maintenance_action                  | 0
date_closing                             | 849
demand_date                              | 571
issue_date                               | 0
issue_apl                                | 0
issue_eic                                | 2
niin                                     | 631
niin_nomenclature                        | 612
unit_of_issue                            | 0
supply_pri                               | 27
source_code                              | 2330
quantity                                 | 0
unit_price                               | 0
job_seq                                  | 0


In [23]:
#niin and niin_nomenclature NA Handling.  Handling these NAs first as there is no sense in imputing values for rows that will be deleted anyway

#631 NAs in niin, 612 NAs in niin_nomenclature

#Looking at these 631 records, the data appears to be unreliable 
#~50% have outlier suddy_days_waiting, another 33% have missing supply days waiting 
#~70% are missing a source_code 
#~70% have issue before demand date

#Because these 631 records appear to be mostly noise/ incorrectly entered, I decided to remove them altogether

data = data[data['niin'].notna()] #remove rows with NA for niin

#1 NA remains for niin_nomenclature. Remove row
data = data[data['niin_nomenclature'].notna()]

In [24]:
#date_closing NA Handling 

#849 NAs
#NA Represents a job that is still open when the data was collected

data['date_closing'] = data['date_closing'] = pd.to_datetime(data['date_closing'], errors = 'coerce') #maintains NAs as NaT


In [25]:
#demand_date NA Handling

#Of the 14330 unique job_seq, there are only 265 NA demand date. when considering the 35194 rows, there are 571 NAs

data['demand_date'] = pd.to_datetime(data['demand_date'], errors = 'coerce') #maintains NAs as NaT

In [26]:
#issue_eic NA Handling 

#2 NAs. Replace with mode
data['issue_eic'] = data['issue_eic'].fillna('T200000')

In [27]:
#supply_pri NA Handling 

#27 NAs. Replace with mode

data['supply_pri'] = data['supply_pri'].fillna(4.0) #data type will be converted later

In [28]:
#source_code NA Handling 
#Refers to item availability at the time of ordering
#2330 NAs. Create new category'X' that will refer to NAs
data['source_code'] = data['source_code'].fillna('X')

# Converting Data Types

In [29]:
#Strings
strings = ['jcn','uic','work_center','jsn', 'issue_apl','issue_eic','niin','niin_nomenclature','job_seq']
for col in strings:
    data[col] = data[col].astype('string')


#Categories
categories = ['unit_of_issue','supply_pri','source_code']
for column in categories:
    data[column] = data[column].astype('category')

#Datetime
data['date_maintenance_action'] = pd.to_datetime(data['date_maintenance_action'], errors = 'coerce')
data['date_closing'] = pd.to_datetime(data['date_closing'], errors = 'coerce')
data['demand_date'] = pd.to_datetime(data['demand_date'], errors = 'coerce')
data['issue_date'] = pd.to_datetime(data['issue_date'], errors = 'coerce')


# Data Types Before and After Modifications
<pre>
Column Name                                  Original Data Type           Data Type After Modification

jcn                                          object                       String                                    
uic                                          int64                        String                                       
work_center                                  object                       String                                    
jsn                                          object                       String                                
date_maintenance_action                      object                       datetime64[ns]                                  
date_closing                                 object                       datetime64[ns]                               
demand_date                                  object                       datetime64[ns]                               
issue_date                                   object                       datetime64[ns]                        
issue_apl                                    object                       String                                    
issue_eic                                    object                       String                                
niin                                         float64                      String                             
niin_nomenclature                            object                       String                                     
unit_of_issue                                object                       Category                                 
supply_pri                                   float64                      Category                                  
source_code                                  object                       Category                                
quantity                                     int64                        Good                                  
unit_price                                   float64                      Good                                 
total_price                                  float64                      Good                                       
job_seq                                      int64                        String                                    


------------------------------------------------------------------------------------------
Added Variables:

issue_before_demand                         None                          bool                                 
supply_days_waiting                         None                          int64  
tot_price                                   None                          float64                                 
supply_days_waiting_outlier                 None                          bool
quantity_outlier                            None                          bool
unit_price_outlier                          None                          bool
<pre>

# Supply Wait Time (Issue Vs. Demand Date) Variable Creation

In [30]:
#Create a flag variable indicating that issue date comes before demand date

#(data['cwt_hours'] < 0).mean() #6.3% of cwt_hours are negative. This is likely due to a data entry error.

data['issue_before_demand'] = (data['issue_date'] < data['demand_date']) #will still have access to this info. Can subset to exclude these rows later if needed

In [31]:
#Create supply_days_waiting: Variable that calculates the number of days between issue date and demand date.

data['supply_days_waiting'] = abs(data['issue_date'] - data['demand_date']) #absolute value assumes the entries were entered in the wrong order
data['supply_days_waiting'] = data['supply_days_waiting'].dt.days #convert to days


# Calculating tot_price Manually

In [32]:
data['tot_price'] = data['quantity'] * data['unit_price']

# Outlier Detection and Handling 

In [33]:
#Handling Outliers for supply_days_waiting

#Create flag variable signaling a Supply Days Waiting Outlier

days_waiting_q1 = data['supply_days_waiting'].quantile(0.25)
days_waiting_q3 = data['supply_days_waiting'].quantile(0.75)
days_waiting_IQR = days_waiting_q3 - days_waiting_q1

data['supply_days_waiting_outlier'] = ((data['supply_days_waiting'] > days_waiting_q3 + 1.5 * days_waiting_IQR)) #Flag variable telling whether supply_days_waiting is an outlier
#days_waiting_outliers = data[(data['supply_days_waiting'] < days_waiting_q1 - 1.5 * days_waiting_IQR) | (data['supply_days_waiting'] > days_waiting_q3 + 1.5 * days_waiting_IQR)] 
#3.6k outliers 

In [34]:
#Handling Outliers for quantity

data['quantity_outlier'] = ((data['quantity'] > 8)) #Flag variable telling whether quantity is an outlier or not for that record

#key: do not want outlier type values to dominate counts

#not using IQR here because of the distribution of this column

In [35]:
#Handling Outliers for unit_price

#Create flag variable signaling a unit_price Outlier

unit_price_q1 = data['unit_price'].quantile(0.25)
unit_price_q3 = data['unit_price'].quantile(0.75)
unit_price_IQR = unit_price_q3 - unit_price_q1

data['unit_price_outlier'] = ((data['unit_price'] > unit_price_q3 + 1.5 * unit_price_IQR)) #Flag variable telling whether unit_price is an outlier
#unit_price_outliers = data[(data['unit_price'] < unit_price_q1 - 1.5 * unit_price_IQR) | (data['unit_price'] > unit_price_q3 + 1.5 * unit_price_IQR)] 
#5.5k outliers 

 


# Notes on Dates For Unique Job Sequences

<pre>
Are days waiting the same for all parts associated with a job_seq or differ?

issue_date and demand_date can differ. There could be up to 20 unique issue_date and demand_date for a given job_seq

date_maintenance_action and date_closing never differ for job_seq
<pre>

In [36]:
# """
# Cell generated by Data Wrangler. #Shows distinct counts of dates for each job_seq
# """
# def clean_data(data):
#     # Performed 4 aggregations grouped on column: 'job_seq'
#     data = data.groupby(['job_seq']).agg(date_maintenance_action_nunique=('date_maintenance_action', 'nunique'), date_closing_nunique=('date_closing', 'nunique'), issue_date_nunique=('issue_date', 'nunique'), demand_date_nunique=('demand_date', 'nunique')).reset_index()
#     return data

# data_clean = clean_data(data.copy())
# data_clean.head()

<pre>
Consider: Job orders could be made for more than 1 job 
For example, an order with 10+ parts likely is not to satisfy only one job.
<pre>

# Simple Date Validity Check

In [37]:
#Are these dates reliable? 
#Reliable: date_maintenace_action <= demand_date <= issue_date <= date_closing

data['valid_dates'] = ((data['date_maintenance_action'] <= data['demand_date']) & (data['demand_date'] <= data['issue_date'])
                         & (data['issue_date'] <= data['date_closing']))

# Writing The Cleaned Dataset

In [38]:
data.to_pickle('Notebooks/jeff-supply-cleaned-2025.pkl') #34562 x 24 (removed 632 rows containing NA niin_nomenclature, niin values)